In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns;
sns.set()

In [2]:
data=pd.read_csv('Melbourne_housing_FULL.csv')
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [3]:
data.shape

(34857, 21)

In [4]:
data.isnull().sum()

Suburb               0
Address              0
Rooms                0
Type                 0
Price             7610
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
YearBuilt        19306
CouncilArea          3
Lattitude         7976
Longtitude        7976
Regionname           3
Propertycount        3
dtype: int64

In [5]:
data_new=data.drop(data[['Address','Method','SellerG','Date', 'Postcode','Lattitude',
                     'Longtitude','Regionname','Propertycount']], axis=1, inplace=True)
data_new.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [6]:
data.columns

Index(['Suburb', 'Rooms', 'Type', 'Price', 'Distance', 'Bedroom2', 'Bathroom',
       'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea'],
      dtype='object')

In [7]:
data.head()

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea
0,Abbotsford,2,h,NaN,2.5,2.0,1.0,1.0,126.0,NaN,NaN,Yarra City Council
1,Abbotsford,2,h,1480000.0,2.5,2.0,1.0,1.0,202.0,NaN,NaN,Yarra City Council
2,Abbotsford,2,h,1035000.0,2.5,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra City Council
3,Abbotsford,3,u,NaN,2.5,3.0,2.0,1.0,0.0,NaN,NaN,Yarra City Council
4,Abbotsford,3,h,1465000.0,2.5,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra City Council


In [8]:
data.columns

Index(['Suburb', 'Rooms', 'Type', 'Price', 'Distance', 'Bedroom2', 'Bathroom',
       'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea'],
      dtype='object')

In [10]:
data.isnull().sum()

Suburb              0
Rooms               0
Type                0
Price            7610
Distance            1
Bedroom2         8217
Bathroom         8226
Car              8728
Landsize        11810
BuildingArea    21115
YearBuilt       19306
CouncilArea         3
dtype: int64

In [11]:
data.shape

(34857, 12)

In [12]:
data.dropna(axis=0, how='any',thresh=None, subset=None, inplace=True)

In [13]:
data.shape

(8895, 12)

In [15]:
features_df=pd.get_dummies(data,columns=['Suburb','CouncilArea','Type'])

In [16]:
#Remove the price column coz it will act as the dependent variable(y)
del features_df['Price']

In [18]:
#create X and y arrays using the matrix data type
X=features_df.values
y=data['Price'].values

#### Split the dataset

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.3,
                                                 random_state=0)

#### select the model and configure its hyperparameters

In [21]:
from sklearn import ensemble

model=ensemble.GradientBoostingRegressor(
 n_estimators=150,
 learning_rate=0.1,
 max_depth=30,
 min_samples_split=4,
 min_samples_leaf=6,
 max_features=0.6,
 loss='huber'
)

In [22]:
model.fit(X_train,y_train)

GradientBoostingRegressor(loss='huber', max_depth=30, max_features=0.6,
                          min_samples_leaf=6, min_samples_split=4,
                          n_estimators=150)

In [25]:
#Save the training model as a file
import sklearn.externals 
import joblib
joblib.dump(model,'house_trained_model.pkl')

['house_trained_model.pkl']

#### Evaluate Results

In [26]:
from sklearn.metrics import mean_absolute_error

mse=mean_absolute_error(y_train,model.predict(X_train))
print("Training Set Mean Absolute Error: %.2f"%mse)

Training Set Mean Absolute Error: 29658.31


In [27]:
mse=mean_absolute_error(y_test,model.predict(X_test))
print("Test Set Mean Absolute Error: %.2f"%mse)

Train Set Mean Absolute Error: 163842.06
